# Plotting Results

Effective visualization is essential for understanding power system dynamics. After running a time-domain simulation, you typically want to examine how variables like generator speeds, bus voltages, and power flows change over time. ANDES provides a built-in plotting interface through `ss.TDS.plt` that makes it easy to create publication-quality figures with minimal code.

This tutorial covers the plotting capabilities available in ANDES, from basic variable plotting to customization options and exporting results. By the end, you will be able to create informative plots that clearly communicate simulation results.

In [ ]:
# Reduce logging verbosity for PDF builds
import os
if os.environ.get('SPHINX_BUILD_PDF'):
    import andes
    _orig_config_logger = andes.config_logger
    def _quiet_logger(stream_level=20, *args, **kwargs):
        stream_level = max(stream_level, 30)
        return _orig_config_logger(stream_level, *args, **kwargs)
    andes.config_logger = _quiet_logger

## Setup and Run Simulation

Before we can plot results, we need simulation data. Let us run a time-domain simulation on the Kundur two-area system, which contains a line trip disturbance that triggers interesting oscillatory dynamics.

In [1]:
%matplotlib inline

import andes

andes.config_logger(stream_level=20)

# Load system and run power flow
ss = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss.PFlow.run()

# Run time-domain simulation
ss.TDS.config.tf = 10
ss.TDS.run()

Working directory: "/Users/hcui7/repos/andes/docs_new/source/tutorials"


> Loaded config from file "/Users/hcui7/.andes/andes.rc"


> Loaded generated Python code in "/Users/hcui7/.andes/pycode".


Parsing input file "/Users/hcui7/repos/andes/andes/cases/kundur/kundur_full.xlsx"...


Input file parsed in 0.2677 seconds.


...

<Toggle 1>: Line.Line_8 status changed to 0 at t=2.0 sec.


Simulation to t=10.00 sec completed in 0.2346 seconds.


Outputs to "kundur_full_out.lst" and "kundur_full_out.npz".


Outputs written in 0.0069 seconds.


True

## The Plotter Object

When running ANDES interactively (in Jupyter Notebook or IPython), a plotter object is automatically created at `ss.TDS.plt` after time-domain simulation completes. This object provides methods for plotting variables, finding variable indices, and exporting data.

You can verify that the plotter was created successfully by checking its type.

In [2]:
ss.TDS.plt

If `ss.TDS.plt` is `None` (which can happen if the interactive environment detection fails), you can manually create it by calling `ss.TDS.load_plotter()`. This step is normally not needed but is useful to know about for troubleshooting.

In [3]:
ss.TDS.load_plotter()

## Plotting by Variable

The most straightforward way to plot simulation results is by passing a variable object directly to the `plot()` method. This approach is intuitive because you reference the exact quantity you want to visualize without needing to look up numerical indices.

For example, to plot all generator rotor speeds (the `omega` variable of the GENROU model), simply pass `ss.GENROU.omega` to the plotter.

In [4]:
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The plot shows the characteristic inter-area oscillations of the Kundur two-area system. After the line trips at t=2s, generators in the same area swing together while generators in different areas swing against each other. This inter-area mode is a key concern in power system stability analysis.

The `plot()` method returns both a figure object and an axis object, which you can use for further customization with matplotlib.

### Plotting a Subset of Variables

When a model has many instances, you may only want to plot a subset rather than all of them. The `a` argument accepts a tuple of zero-indexed positions indicating which variables to include.

For example, to plot only the first and fourth generators (positions 0 and 3), use:

In [5]:
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega, a=(0, 3))

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


You can also plot bus voltage magnitudes. The bus voltage variable `v` is accessed through `ss.Bus.v` (note that this is an algebraic variable rather than a state variable like generator speed).

In [6]:
fig, ax = ss.TDS.plt.plot(ss.Bus.v)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Index-Based Plotting

For more advanced use cases or when working with the command-line plotting tool, you may need to plot by variable index rather than by variable object. ANDES assigns each variable a unique index that is recorded in the output `.lst` file.

The `find()` method helps you discover these indices. It searches variable names using patterns (with regular expression support) and returns both the indices and the matching variable names.

In [7]:
# Find all variables containing 'omega'
ss.TDS.plt.find('omega')

([5, 6, 7, 8],
 ['omega GENROU 1', 'omega GENROU 2', 'omega GENROU 3', 'omega GENROU 4'])

Once you know the indices, you can pass them as a tuple or list to `plot()`. This produces the same result as plotting by variable but uses the numerical indices instead.

In [8]:
# Plot using indices directly
fig, ax = ss.TDS.plt.plot((5, 6, 7, 8))

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The index-based approach is particularly useful when using the `andes plot` command from the terminal:

```bash
andes plot kundur_full_out.lst 0 5 6 7 8
```

Here, index 0 represents time, and indices 5-8 represent the generator speeds.

## Customizing Plots

The `plot()` method accepts numerous arguments for customizing the appearance of your figures. These options let you create publication-ready plots without needing to write additional matplotlib code.

### Labels and Legends

In [9]:
fig, ax = ss.TDS.plt.plot(
    ss.GENROU.omega,
    ylabel='Generator Speed [pu]',
    yheader=['Gen 1', 'Gen 2', 'Gen 3', 'Gen 4']
)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


The `ylabel` argument sets the y-axis label, while `yheader` provides custom legend entries. When using `yheader`, ensure the number of entries matches the number of variables being plotted.

### Scaling Values

Power system variables are often stored in per-unit but may be more intuitive when displayed in other units. The `ycalc` argument accepts a callable that transforms the y-values before plotting.

For example, to convert generator speed from per-unit to Hz (for a 60 Hz system where 1.0 pu = 60 Hz):

In [10]:
fig, ax = ss.TDS.plt.plot(
    ss.GENROU.omega,
    ycalc=lambda x: 60 * x,
    ylabel='Frequency [Hz]'
)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Grid and Greyscale

For technical publications or presentations, you may want to add a grid for easier reading or use greyscale for black-and-white printing.

In [11]:
fig, ax = ss.TDS.plt.plot(
    ss.GENROU.omega,
    ycalc=lambda x: 60 * x,
    ylabel='Frequency [Hz]',
    grid=True,
    greyscale=True
)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


### Adding Curves to an Existing Figure

When comparing results from different scenarios or showing related variables on the same plot, you can add new curves to an existing figure. Pass the `fig` and `ax` objects from a previous plot to overlay new data.

In [12]:
# First plot: generators 1 and 4
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega, a=(0, 3))

# Add generator 2 with a different line style
fig, ax = ss.TDS.plt.plot(
    ss.GENROU.omega,
    a=(1,),
    fig=fig,
    ax=ax,
    linestyles=['-.']
)

/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Saving Figures

Figures can be saved directly from the `plot()` method using the `savefig` argument. When set to `True`, the figure is saved with an automatically generated filename. You can also pass a string to specify the exact filename.

In [13]:
fig, ax = ss.TDS.plt.plot(
    ss.GENROU.omega,
    ylabel='Generator Speed [pu]',
    savefig='generator_speeds.png'
)

Figure saved to "generator_speeds.png.png".


/Users/hcui7/repos/andes/andes/plot.py:822: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


For more control over the output format and resolution, you can save the figure using matplotlib directly. This is useful when you need vector formats like PDF for publications.

In [14]:
fig, ax = ss.TDS.plt.plot(ss.GENROU.omega, show=False)
ax.set_title('Generator Response to Line Trip')
ax.set_ylabel('Speed [pu]')
ax.grid(True)
fig.savefig('generator_response.pdf', dpi=300, bbox_inches='tight')

## Exporting to CSV

While plots are useful for visualization, you often need the raw numerical data for further analysis in spreadsheets or other tools. The `export_csv()` method exports all simulation results to a CSV file that can be opened in Excel or read by other programs.

In [15]:
ss.TDS.plt.export_csv('simulation_results.csv')

CSV data saved to "simulation_results.csv".


'simulation_results.csv'

For more selective export, you can access the underlying data arrays directly and create a custom DataFrame. The time series data is stored in `ss.dae.ts`, with `t` for time, `x` for state variables, and `y` for algebraic variables.

In [16]:
import pandas as pd

# Create DataFrame with selected variables
data = {
    'time': ss.dae.ts.t,
    'Gen1_omega': ss.dae.ts.x[:, ss.GENROU.omega.a[0]],
    'Gen2_omega': ss.dae.ts.x[:, ss.GENROU.omega.a[1]],
    'Bus1_v': ss.dae.ts.y[:, ss.Bus.v.a[0]],
}

df = pd.DataFrame(data)
df.head()

,time,Gen1_omega,Gen2_omega,Bus1_v
0,0.000000,1.0,1.0,1.0
1,0.033333,1.0,1.0,1.0
2,0.066667,1.0,1.0,1.0
3,0.100000,1.0,1.0,1.0
4,0.133333,1.0,1.0,1.0


In [17]:
df.to_csv('selected_results.csv', index=False)

## Plot Method Reference

The `plot()` method supports many additional arguments for fine-grained control. Here is a summary of the most commonly used options:

| Argument | Description |
|----------|-------------|
| `a` | Tuple of indices to select subset of variables |
| `ylabel` | Y-axis label |
| `yheader` | List of legend entries |
| `ycalc` | Callable to transform y-values |
| `grid` | Show grid lines (True/False) |
| `greyscale` | Use greyscale colors (True/False) |
| `savefig` | Save figure (True or filename) |
| `dpi` | Resolution for saved figure |
| `left`, `right` | X-axis limits |
| `ymin`, `ymax` | Y-axis limits |
| `fig`, `ax` | Existing figure/axis to plot on |
| `linestyles` | List of line styles |
| `line_width` | Width of plot lines |
| `font_size` | Size of labels and legends |

## Cleanup

In [18]:
!andes misc -C
!rm -f generator_speeds.png generator_response.pdf simulation_results.csv selected_results.csv

"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.txt" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.npz" removed.
"/Users/hcui7/repos/andes/docs_new/source/tutorials/kundur_full_out.lst" removed.


## Next Steps

- {doc}`07-eigenvalue-analysis` - Small-signal stability analysis
- {doc}`08-parameter-sweeps` - Running batch simulations
- {doc}`09-contingency-analysis` - N-1 contingency screening